In [1]:
import librosa 
import numpy as np
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [2]:
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = '../dataset/audio/'

#metadata = pd.read_csv('../dataset/dataset.csv')
metadata = pd.read_csv('../dataset/datasetM2.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class_name"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files') 

Finished feature extraction from  13242  files


In [3]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

Using TensorFlow backend.


In [4]:
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [5]:
### store the preprocessed data for use in the next notebook

%store x_train 
%store x_test 
%store y_train 
%store y_test 
%store yy 
%store le

Stored 'x_train' (ndarray)
Stored 'x_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)
Stored 'yy' (ndarray)
Stored 'le' (LabelEncoder)


In [6]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [8]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [9]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

In [10]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 10593 samples, validate on 2649 samples
Epoch 1/100
10593/10593 [==============================] - 1s 127us/step - loss: 8.1442 - accuracy: 0.2582 - val_loss: 1.5610 - val_accuracy: 0.3043

Epoch 00001: val_loss improved from inf to 1.56099, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/100
10593/10593 [==============================] - 1s 117us/step - loss: 1.5849 - accuracy: 0.3080 - val_loss: 1.4674 - val_accuracy: 0.3915

Epoch 00002: val_loss improved from 1.56099 to 1.46738, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 3/100
10593/10593 [==============================] - 1s 114us/step - loss: 1.4838 - accuracy: 0.3444 - val_loss: 1.3717 - val_accuracy: 0.4292

Epoch 00003: val_loss improved from 1.46738 to 1.37171, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/100
10593/10593 [==============================] - 1s 126us/step - loss: 1.3647 - accuracy: 0.3982 - val_loss: 1.1813 - val_accuracy: 0.5364

Epoch 00004: val

10593/10593 [==============================] - 1s 106us/step - loss: 0.4072 - accuracy: 0.8537 - val_loss: 0.2172 - val_accuracy: 0.9305

Epoch 00037: val_loss did not improve from 0.01383
Epoch 38/100
10593/10593 [==============================] - 1s 109us/step - loss: 0.3950 - accuracy: 0.8626 - val_loss: 0.1022 - val_accuracy: 0.9724

Epoch 00038: val_loss did not improve from 0.01383
Epoch 39/100
10593/10593 [==============================] - 1s 106us/step - loss: 0.3002 - accuracy: 0.9022 - val_loss: 0.1413 - val_accuracy: 0.9407

Epoch 00039: val_loss did not improve from 0.01383
Epoch 40/100
10593/10593 [==============================] - 1s 106us/step - loss: 0.3491 - accuracy: 0.8811 - val_loss: 0.1314 - val_accuracy: 0.9536

Epoch 00040: val_loss did not improve from 0.01383
Epoch 41/100
10593/10593 [==============================] - 1s 116us/step - loss: 0.3845 - accuracy: 0.8692 - val_loss: 0.1472 - val_accuracy: 0.9407

Epoch 00041: val_loss did not improve from 0.01383
Epo

10593/10593 [==============================] - 2s 148us/step - loss: 0.6198 - accuracy: 0.7773 - val_loss: 0.6560 - val_accuracy: 0.8256

Epoch 00077: val_loss did not improve from 0.01383
Epoch 78/100
10593/10593 [==============================] - 1s 140us/step - loss: 0.6494 - accuracy: 0.7592 - val_loss: 0.7347 - val_accuracy: 0.7928

Epoch 00078: val_loss did not improve from 0.01383
Epoch 79/100
10593/10593 [==============================] - 2s 144us/step - loss: 0.6346 - accuracy: 0.7708 - val_loss: 0.6011 - val_accuracy: 0.8743

Epoch 00079: val_loss did not improve from 0.01383
Epoch 80/100
10593/10593 [==============================] - 1s 136us/step - loss: 0.6015 - accuracy: 0.7804 - val_loss: 0.5271 - val_accuracy: 0.8743

Epoch 00080: val_loss did not improve from 0.01383
Epoch 81/100
10593/10593 [==============================] - 1s 130us/step - loss: 0.5753 - accuracy: 0.7913 - val_loss: 0.6457 - val_accuracy: 0.8743

Epoch 00081: val_loss did not improve from 0.01383
Epo

In [11]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.5815160870552063
Testing Accuracy:  0.6043789982795715


In [12]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])


In [13]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [14]:
# test

filename = '../dataset/audio/whatsappTest.wav' 
print_prediction(filename) 

The predicted class is: zoom 

double_tap 		 :  0.01401440799236297607421875000000
long_tap 		 :  0.22350324690341949462890625000000
swipe 		 :  0.07667756825685501098632812500000
tap 		 :  0.04647029563784599304199218750000
zoom 		 :  0.63933449983596801757812500000000
